In [5]:
import numpy as np
from utils_clean import generate_sample, DEOptim, RandomOptim, PSOOptim, visualize_meshgrid, get_likelihood_grids, visualize_pred_meshgrid
from sklearn.gaussian_process.kernels import RBF
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
import plotly.express as px
import pandas as pd

In [6]:
target_func = lambda X: (20 + X**2 - 10 * np.cos(2*np.pi*X)).sum(axis=0)

n_dims = 2
lower_bound = -2.5, -1.5
upper_bound = +1.5, +2.5

In [7]:
de = DEOptim(visualize=False, maxiter=100, popsize=10)

In [8]:
rs = RandomOptim(maxiter=1000, visualize=False)

In [9]:
seed = np.random.RandomState(42)
part = 10
dim = 2
pso = PSOOptim(n_iters=100, n_particles=part, c1=0.5, c2=0.3, w=0.9, init_pos=(seed.rand(part, dim)*22)-11, visualize=False)

In [10]:
results_dict = {
    "optim": [],
    "n": [],
    "log_likelihood": [],
    "MSE": []
}

In [11]:

for n in [10, 50, 200, 500, 1000, 5000, 10000]:
    for optim in [de, rs, pso]:
        gpr = GaussianProcessRegressor(optimizer=optim.optimize, kernel=RBF(length_scale=(1,1)))
        X, y = generate_sample(n, n_dims, lower_bound, upper_bound, target_func)
        gpr.fit(X.T, y[...,None])
        results_dict["optim"].append(optim.__class__.__name__)
        results_dict["n"].append(n)
        results_dict["log_likelihood"].append(gpr.log_marginal_likelihood_value_)
        results_dict["MSE"].append(mean_squared_error(y, gpr.predict(X.T)))

2022-08-08 09:38:50,573 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=5.32e+3
2022-08-08 09:38:51,022 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 5315.410680369145, best pos: [ 1.38358591 -0.96097825]
2022-08-08 09:38:51,913 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=2.66e+4
2022-08-08 09:38:52,748 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 26606.782010108942, best pos: [-1.01880562 -0.90669974]
2022-08-08 09:38:58,292 - pyswarms.single.global_best - INFO - Optimize for 100 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████|100/100, best_cost=4.57e+4
2022-08-08 09:39:01,844 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 45663.9

KeyboardInterrupt: 

In [12]:
df = pd.DataFrame(results_dict)

In [13]:
df

,optim,n,log_likelihood,MSE
0,DEOptim,10,-5315.400923,7.310157e-15
1,RandomOptim,10,-6126.819983,2.315093e-14
2,PSOOptim,10,-5315.410680,7.378864e-15
3,DEOptim,50,-26606.716685,3.841156e-17
4,RandomOptim,50,-31657.669063,5.184422e-17
5,PSOOptim,50,-26606.782010,3.869805e-17
6,DEOptim,200,-45663.230583,1.095328e-15
7,RandomOptim,200,-60889.464920,1.165845e-16
8,PSOOptim,200,-45663.960528,1.064659e-15
9,DEOptim,500,-49515.693722,1.344786e-11


In [14]:
px.line(df, x="n", y="log_likelihood", color="optim")

In [15]:
px.line(df, x="n", y="MSE", color="optim")